#### original notebook: https://www.kaggle.com/riow1983/m5-three-shades-of-dark-darker-magic/edit

このNotebookはYakovlevの [M5 - Three shades of Dark: Darker magic](https://www.kaggle.com/kyakovlev/m5-three-shades-of-dark-darker-magic) を研究するために作ったものをShareしています。

Thank Yakovlev for sharing very much.

次のNotebookにも依存していることに注意しましょう。<br>
[【日本語】M5 - Simple FE](https://www.kaggle.com/ejunichi/m5-simple-fe)<br>
[M5 - Custom features](https://www.kaggle.com/kyakovlev/m5-custom-features)<br>
[M5 - Lags features](https://www.kaggle.com/kyakovlev/m5-lags-features)　

In [2]:
#!pip install psutil

     |████████████████████████████████| 449 kB 2.7 MB/s eta 0:00:01
  Created wheel for psutil: filename=psutil-5.7.0-cp37-cp37m-linux_x86_64.whl size=261186 sha256=b6fc11c3273aef57e1713958c3ea07bfffdae9be98849ac983fed7bf448a6a08
  Stored in directory: /root/.cache/pip/wheels/b6/e7/50/aee9cc966163d74430f13f208171dee22f11efa4a4a826661c
Successfully built psutil


In [3]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [4]:
########################### ヘルパー
#################################################################################
## シード生成
# ：すべてのプロセスを確定的にするためのシード     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## マルチプロセス実行
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [5]:
########################### store IDによってデータをロードするヘルパー
#################################################################################
# データ読込
def get_data_by_store(store):
    
    # 基本特徴量を読んで連絡する
    # BASE,PRICE,CALENDARは「【日本語】M5 - Simple FE」を参照
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # 関連する店舗のみを残す
    df = df[df['store_id']==store]

    # メモリの制限があるため、LAGとエンコード特徴量を個別に読み取り、
    # 不要なアイテムを削除する必要があります。 
    # 特徴量グリッドが整列されるため、必要な行のみを保持するために index を使用できます
    # concat は merge よりも少ないメモリを使用するため、整列は適切です。
    df2 = pd.read_pickle(MEAN_ENC)[mean_features] # 「M5 - Custom features」を参照
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:] # 「M5 - Lags features」を参照
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # メモリ制限に達しないように削除
    
    df = pd.concat([df, df3], axis=1)
    del df3 # メモリ制限に達しないように削除
    
    # 特徴量リストを作成する
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # 最初のn行をスキップ
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# トレーニング後にテストを再結合
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### 動的移動LAGを作成するヘルパー
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [6]:
########################### モデルパラメータ
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

In [7]:
########################### 変数
#################################################################################
VER = 1                          # バージョン
SEED = 42                        # 
seed_everything(SEED)            # 私たちはすべてのものが出来るだけ
lgb_params['seed'] = SEED        # 確定的であることを望みます。
N_CORES = psutil.cpu_count()     # 使用可能なCPUコア


#限界と定数
TARGET      = 'sales'            # ターゲット
START_TRAIN = 0                  # 一部の行をスキップできます(Nans/faster training)
END_TRAIN   = 1913               # trainの終了日
P_HORIZON   = 28                 # 予測期間
USE_AUX     = True               # 事前学習済みモデルを使用するかどうか

#削除する特徴量
## これらの機能はオーバーフィットにつながります 
## または test に存在しない値
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#特徴量のパス
ORIGINAL = '../input/m5-forecasting-accuracy/'
BASE     = '../input/m5-simple-fe/grid_part_1.pkl'
PRICE    = '../input/m5-simple-fe/grid_part_2.pkl'
CALENDAR = '../input/m5-simple-fe/grid_part_3.pkl'
LAGS     = '../input/m5-lags-features/lags_df_28.pkl'
MEAN_ENC = '../input/m5-custom-features/mean_encoding_df.pkl'


# AUX（事前学習済み）モデルのパス
AUX_MODELS = '../input/m5-aux-models/'


#STORES id
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#LAG作成用の分割
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

FileNotFoundError: [Errno 2] File ../input/m5-forecasting-accuracy/sales_train_validation.csv does not exist: '../input/m5-forecasting-accuracy/sales_train_validation.csv'

In [ ]:
########################### AUXモデル

if USE_AUX:
    lgb_params['n_estimators'] = 2
    
# ここに比較できるいくつかの「ログ」があります
#Train CA_1
#[100]	valid_0's rmse: 2.02289
#[200]	valid_0's rmse: 2.0017
#[300]	valid_0's rmse: 1.99239
#[400]	valid_0's rmse: 1.98471
#[500]	valid_0's rmse: 1.97923
#[600]	valid_0's rmse: 1.97284
#[700]	valid_0's rmse: 1.96763
#[800]	valid_0's rmse: 1.9624
#[900]	valid_0's rmse: 1.95673
#[1000]	valid_0's rmse: 1.95201
#[1100]	valid_0's rmse: 1.9476
#[1200]	valid_0's rmse: 1.9434
#[1300]	valid_0's rmse: 1.9392
#[1400]	valid_0's rmse: 1.93446

#Train CA_2
#[100]	valid_0's rmse: 1.88949
#[200]	valid_0's rmse: 1.84767
#[300]	valid_0's rmse: 1.83653
#[400]	valid_0's rmse: 1.82909
#[500]	valid_0's rmse: 1.82265
#[600]	valid_0's rmse: 1.81725
#[700]	valid_0's rmse: 1.81252
#[800]	valid_0's rmse: 1.80736
#[900]	valid_0's rmse: 1.80242
#[1000]	valid_0's rmse: 1.79821
#[1100]	valid_0's rmse: 1.794
#[1200]	valid_0's rmse: 1.78973
#[1300]	valid_0's rmse: 1.78552
#[1400]	valid_0's rmse: 1.78158

In [ ]:
########################### モデルのトレーニング
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # 現在の店舗のグリッドを取得
    grid_df, features_columns = get_data_by_store(store_id)
    
    # マスク
    # Train （1913未満のすべてのデータ）
    # "Validation" （過去28日間-実際の検証セットではない）
    # Test （1913日を超えるすべてのデータ、再帰的特徴量のギャップあり）
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # マスクを適用してlgbデータセットをbinとして保存し、
    # dtype変換中のメモリスパイクを削減する
    # https://github.com/Microsoft/LightGBM/issues/1032
    # 変換を回避するには、常に np.float32 を使用するか、
    # トレーニングを開始する前に bin に保存する必要があります
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # 後の予測のためにデータセットの一部を保存する
    # 再帰的に計算する必要がある特徴量を削除する
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # シーダーを再度起動して、
    # 各「コード行」np.randomが「進化」するlgbトレーニングを100％確定的にするため、
    # 「リセット」する必要がある場合があります。
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    # モデルを保存-実際の「.bin」ではなく
    # estimator = lgb.Booster(model_file='model.txt')
    # pickleファイルとすると最良の反復（または保存反復）でのみ予測できます
    # pickle.dumpは柔軟性を提供します。
    # estimator.predict(TEST, num_iteration=100)
    # num_iteration - 予測したい反復回数
    # NULLまたは<= 0は最適な反復を使用することを意味します
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # 一時ファイルとオブジェクトを削除して、
    # HDDスペースとRAMメモリを解放します
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # 予測のためのモデル特徴量の保持
    MODEL_FEATURES = features_columns

In [ ]:
########################### 予測
#################################################################################

# 予測を保存するためのダミーデータフレームを作成する
all_preds = pd.DataFrame()

# テストデータセットをトレーニングデータの一部に結合して、
# 再帰的な特徴量を作成します
base_test = get_base_test()

# 予測時間を測定するタイマー
main_time = time.time()

# 各予測日にループする移動LAGは最も時間がかかるため、
# 1日全体で計算します。
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # 移動LAGを計算するための一時的なグリッドを作成する
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # すべてのモデルを読んで、日/店舗のペアごとに予測を行う
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # 適切な列の名前を付け、all_preds に追加します
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

In [ ]:
########################### 書き出す
#################################################################################
# コンぺのサンプル提出を読んで、予測を結合する
# 「_validation」データのみの予測があるため
# 「_evaluation」アイテムに対してfillna（）を実行する必要がある
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)